In [2]:
import pandas as pd
import random
import sys
sys.path.append("..")
from src.recommenders.online_learner import OnlineNewsRecommender, load_user_history_json, load_news_df

In [9]:
news_df=pd.read_csv("cleaned_data.csv")
user_history_dict = load_user_history_json('../data/processed/user_history.json')

In [10]:
type(user_history)

list

In [11]:
model = OnlineNewsRecommender()

In [13]:
# Cell 4: Simulate feedback loop
n_iterations = 100
users = list(user_history_dict.keys())
accuracies = []

for i in range(n_iterations):
    user = random.choice(users)
    user_history = user_history_dict[user]

    # Learn from user history
    for history in user_history:
        model.learn_one(history, True)
        model.update_metric(history, True)

    # Recommend for user
    candidate_df = news_df.sample(n=10, random_state=i)
    top_k_news_ids = model.recommend_top_k(
        user_history_texts=user_history,
        candidate_news_df=candidate_df,
        k=3
    )

    # Simulate feedback: check if any history word appears in the candidate title+abstract
    for news_id in top_k_news_ids:
        news_row = candidate_df[candidate_df['news_id'] == news_id].iloc[0]
        text = news_row['title'] + " " + str(news_row.get('abstract', ''))

        # Simulated user click if overlap in words
        relevant = any(word in text for h in user_history for word in h.split())
        model.learn_one(text, relevant)
        model.update_metric(text, relevant)

    # Log accuracy
    accuracies.append(model.get_metric())

AttributeError: 'dict' object has no attribute 'encode'